In [44]:
import pandas as pd
import numpy as np
from scipy.sparse import hstack
from sklearn.feature_extraction import FeatureHasher

In [2]:
import os
dirpath = 'cat-in-the-dat'
os.listdir(dirpath)

['test.csv', 'train.csv', 'sample_submission.csv']

In [3]:
data = pd.read_csv(f'{dirpath}/train.csv')

In [4]:
data

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0
5,5,0,1,1,T,N,Blue,Polygon,Lion,Costa Rica,...,51e27c16d,1,Novice,Freezing,j,E,PZ,2,2,0
6,6,0,1,1,T,N,Green,Trapezoid,Cat,China,...,7e3d79a0d,2,Grandmaster,Lava Hot,g,P,wy,5,4,0
7,7,1,0,1,T,Y,Red,Triangle,Dog,Russia,...,feb72ecc2,1,Novice,Lava Hot,j,K,Ed,4,2,0
8,8,1,0,1,T,Y,Blue,Square,Hamster,Canada,...,34a7273bf,2,Novice,Boiling Hot,e,V,qo,3,4,0
9,9,0,0,0,F,Y,Red,Trapezoid,Lion,China,...,0ece7a511,1,Expert,Freezing,h,Q,CZ,3,2,0


In [5]:
trainY = data.iloc[:,-1]
binaryX = data.iloc[:,1:8]
categoryX = data.iloc[:,8:17]
cyclicX = data.iloc[:,17:24]

In [37]:
cyclicX.shape

(300000, 7)

In [32]:
trainX_hash = data.iloc[:,1:24]
trainX = trainX_hash.copy()
for c in trainX.columns:
    trainX_hash[c] = trainX[c].astype('str')
hashTable = FeatureHasher(input_type='string')
train_X_old = hashTable.transform(trainX_hash.values)

In [45]:
train_X_old.shape

array([ 1.,  1.,  1., ..., -1.,  1.,  1.])

In [41]:
binaryX_hash = binaryX.copy()
categoryX_hash = categoryX.copy()
cyclicX_hash = cyclicX.copy()

for c in binaryX.columns:
    binaryX_hash[c] = binaryX[c].astype('str')
binaryTable = FeatureHasher(input_type='string')
binary_X = binaryTable.transform(binaryX_hash.values)

for d in categoryX.columns:
    categoryX_hash[d] = categoryX[d].astype('str')
categoryTable = FeatureHasher(input_type='string')
category_X = categoryTable.transform(categoryX_hash.values)

for e in cyclicX.columns:
    cyclicX_hash[e] = cyclicX[e].astype('str')
cyclicTable = FeatureHasher(input_type='string')
cyclic_X = cyclicTable.transform(cyclicX_hash.values)

In [70]:
# train_X = hstack((binary_X, category_X, cyclic_X))
train_X = ((binary_X+ category_X+ cyclic_X))

In [71]:
train_X.shape

(300000, 1048576)

In [ ]:
# trainX, trainY = data.iloc[:,1:24], data.iloc[:,-1]

In [18]:
train_X

<300000x3145728 sparse matrix of type '<class 'numpy.float64'>'
	with 6441076 stored elements in COOrdinate format>

In [19]:
trainY

0         0
1         0
2         0
3         1
4         0
         ..
299995    0
299996    0
299997    1
299998    1
299999    0
Name: target, Length: 300000, dtype: int64

In [72]:
from sklearn.linear_model import LogisticRegression as lg
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(train_X, trainY, test_size=0.3, random_state=42)

clf = lg()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [73]:
accuracy_score(y_test,y_pred)

0.7503555555555556

In [21]:
accuracy_score(trainY, y_pred)

0.77884

In [28]:
testX = pd.read_csv(f'{dirpath}/test.csv')
testX_hash = testX.copy()

# for c in testX.columns:
#     testX_hash[c] = testX[c].astype('str')
# testHashTable = FeatureHasher(input_type='string')
# test_X = testHashTable.transform(testX_hash.values)

In [47]:
binaryTX = testX.iloc[:,1:8]
categoryTX = testX.iloc[:,8:17]
cyclicTX = testX.iloc[:,17:24]

binaryTX_hash = binaryTX.copy()
categoryTX_hash = categoryTX.copy()
cyclicTX_hash = cyclicTX.copy()

for c in binaryTX.columns:
    binaryTX_hash[c] = binaryTX[c].astype('str')
binaryTable = FeatureHasher(input_type='string')
binaryT_X = binaryTable.transform(binaryTX_hash.values)

for d in categoryTX.columns:
    categoryTX_hash[d] = categoryTX[d].astype('str')
categoryTable = FeatureHasher(input_type='string')
categoryT_X = categoryTable.transform(categoryTX_hash.values)

for e in cyclicTX.columns:
    cyclicTX_hash[e] = cyclicX[e].astype('str')
cyclicTable = FeatureHasher(input_type='string')
cyclicT_X = cyclicTable.transform(cyclicTX_hash.values)

In [75]:
# testStack = hstack((binaryT_X, categoryT_X, cyclicT_X))
testStack = ((binaryT_X+categoryT_X+ cyclicT_X))

In [76]:
test_target = clf.predict_proba(testStack)[:,1]

In [77]:
test_target

array([0.26726621, 0.52236772, 0.06447717, ..., 0.88086736, 0.08755125,
       0.28276493])

In [78]:
submission = pd.read_csv(f'{dirpath}/sample_submission.csv')

submission['target'] = test_target
submission.to_csv('Hashing model.csv', index=False)

In [ ]:
submission

In [ ]:
test_X

In [59]:
print(cyclicT_X)

  (0, 22790)	1.0
  (0, 200276)	1.0
  (0, 433202)	1.0
  (0, 647703)	2.0
  (0, 802906)	-1.0
  (0, 871725)	-1.0
  (1, 354738)	1.0
  (1, 433202)	1.0
  (1, 689704)	1.0
  (1, 694762)	-1.0
  (1, 811797)	1.0
  (1, 849870)	1.0
  (1, 917481)	-1.0
  (2, 118856)	-1.0
  (2, 222657)	1.0
  (2, 540998)	-1.0
  (2, 634453)	1.0
  (2, 647703)	1.0
  (2, 689704)	1.0
  (2, 871725)	-1.0
  (3, 22790)	1.0
  (3, 282075)	1.0
  (3, 364501)	-1.0
  (3, 433202)	1.0
  (3, 550420)	1.0
  :	:
  (199996, 460600)	-1.0
  (199996, 550420)	1.0
  (199996, 611181)	-1.0
  (199996, 930680)	-1.0
  (199997, 205902)	-1.0
  (199997, 296051)	1.0
  (199997, 426177)	-1.0
  (199997, 500148)	1.0
  (199997, 540998)	-1.0
  (199997, 634453)	1.0
  (199997, 795355)	1.0
  (199998, 98813)	-1.0
  (199998, 241575)	-1.0
  (199998, 426177)	-1.0
  (199998, 540998)	-1.0
  (199998, 611181)	-1.0
  (199998, 834929)	1.0
  (199998, 930680)	-1.0
  (199999, 154159)	-1.0
  (199999, 171236)	1.0
  (199999, 500148)	1.0
  (199999, 634453)	1.0
  (199999, 699750)	1

In [61]:
fullT_X = cyclicT_X+categoryT_X+binaryT_X

In [63]:
type(fullT_X)

scipy.sparse.csr.csr_matrix

In [66]:
fullT_X.shape[0]

200000